In [1]:
# Change directory to the root of the project
import os 
os.chdir('..')
os.chdir('..')
os.chdir('..')
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes


In [2]:
from applications.dots_and_boxes.NNmodels.MLP import MLPInitParams
import torch

# Initialize parameters

## Model parameters
model_type = 'mlp'
model_params: MLPInitParams = {
    'num_rows': 5,
    'num_cols': 5,
    'hidden_sizes': [64, 128, 256, 512, 256, 128, 64]
}
device = torch.device('mps')
model_name = 'dots_and_boxes_mlp'

## Initialize new model
load_model = None
load_model_params = {}

## Optimizer parameters
optimizer_type = 'adam'
optimizer_params = {
    'lr': 1e-2,
    'betas': (0.9, 0.999),
    'eps': 1e-8,
    'weight_decay': 1e-4,
    'amsgrad': False
}

## Learning scheduler parameters
lr_scheduler_type = 'plateau'
lr_scheduler_params = {
    'factor': 0.5,
    'patience': 1000,
    'cooldown': 1000,
    'min_lr': 1e-4
}

## Training parameters
training_method = 'supervised'
trainer_params = {}
training_params = {
    'epochs': 200,
    'batch_size': 256,
    'eval_freq': 25,
    'checkpoint_freq': 50,
    'mask_illegal_moves': False,
    'mask_value': -20.0, # Doesn't matter when mask_illegal_moves is False
    'checkpoint_dir': 'checkpoints',
    'start_at': 1
}

## Load replay buffer from wandb
load_replay_buffer = 'from_file'
buffer_type = 'mcts' # used in path name below
load_replay_buffer_params = {
    'path': f'applications/dots_and_boxes/training_data/dots_and_boxes_{model_params["num_rows"]}x{model_params["num_cols"]}_DABSimpleTensorMapping_{buffer_type}.pkl',
    'device': device
}
# load_replay_buffer_params = {
#     'project': 'AlphaZero-DotsAndBoxes',
#     'artifact_name': f'dots_and_boxes_{model_params["num_rows"]}x{model_params["num_cols"]}_SimpleTensorMapping_minimax',
#     'artifact_version': 'latest'
# }


In [3]:
# Initialize wandb run
import wandb

run_name = 'MLP 7'
notes = 'Larger than v6.'

config = {
    'model_type': model_type,
    'model_params': model_params,
    'optimizer_type': optimizer_type,
    'optimizer_params': optimizer_params,
    'lr_scheduler_type': lr_scheduler_type,
    'lr_scheduler_params': lr_scheduler_params,
    'training_method': training_method,
    'trainer_params': trainer_params,
    'training_params': training_params
}

run = wandb.init(
    project='AlphaZero-DotsAndBoxes',
    name=run_name,
    config=config,
    notes=notes,
    group=f'{training_method} training on {model_params["num_rows"]}x{model_params["num_cols"]} board'
)
# run = None

wandb: Currently logged in as: eohjelle (eigenway) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [4]:
# Perform training

from applications.dots_and_boxes.train import train

model_interface = train(
    num_rows = model_params["num_rows"],
    num_cols = model_params["num_cols"],
    model_type=model_type,
    model_params=model_params,
    device=device,
    model_name=model_name,
    optimizer_type=optimizer_type,
    optimizer_params=optimizer_params,
    lr_scheduler_type=lr_scheduler_type,
    lr_scheduler_params=lr_scheduler_params,
    training_method=training_method,
    trainer_params=trainer_params,
    training_params=training_params,
    load_model=load_model,
    load_model_params=load_model_params,
    load_replay_buffer=load_replay_buffer,
    load_replay_buffer_params=load_replay_buffer_params,
    wandb_run=run
)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes/core/data_structures.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to

Epoch 1/200, Train Loss: 3.9028, Val Loss: 3.5593, Learning rate: 0.010000
Epoch 2/200, Train Loss: 3.3662, Val Loss: 3.3097, Learning rate: 0.010000
Epoch 3/200, Train Loss: 3.7432, Val Loss: 4.3139, Learning rate: 0.010000
Epoch 4/200, Train Loss: 4.2698, Val Loss: 4.2166, Learning rate: 0.010000
Epoch 5/200, Train Loss: 4.1739, Val Loss: 4.1481, Learning rate: 0.010000
Epoch 6/200, Train Loss: 4.1185, Val Loss: 4.0677, Learning rate: 0.010000
Epoch 7/200, Train Loss: 3.9715, Val Loss: 3.9486, Learning rate: 0.010000
Epoch 8/200, Train Loss: 3.8526, Val Loss: 3.8522, Learning rate: 0.010000
Epoch 9/200, Train Loss: 3.7748, Val Loss: 3.7888, Learning rate: 0.010000
Epoch 10/200, Train Loss: 3.7289, Val Loss: 3.7607, Learning rate: 0.010000
Epoch 11/200, Train Loss: 3.6711, Val Loss: 3.7090, Learning rate: 0.010000
Epoch 12/200, Train Loss: 3.6064, Val Loss: 3.6481, Learning rate: 0.010000
Epoch 13/200, Train Loss: 3.5498, Val Loss: 3.6176, Learning rate: 0.010000
Epoch 14/200, Train L

In [5]:
run.finish()

RandomAgent_draw_rate,▁▁▁▁▁▁▁▁
RandomAgent_loss_rate,▄█▁▄▇▂▇▄
RandomAgent_score,▅▁█▅▂▇▂▅
RandomAgent_win_rate,▅▁█▅▂▇▂▅
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇██████
learning_rate,█████▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_loss,█▆▆▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_policy_loss,██▆▄▄▄▃▃▃▃▃▃▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
train_value_loss,█▆▅▅▄▄▄▄▃▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▂▁▂▂▁▁▁▁
val_loss,▅█▇▇▆▄▄▄▃▃▃▃▃▃▂▂▂▂▂▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_policy_loss,█▆▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [6]:
def print_model_parameters(model):
    """
    Print the total number of parameters in a PyTorch model,
    with a breakdown of trainable vs non-trainable parameters.
    
    Args:
        model: PyTorch model
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%})")
    print(f"Non-trainable parameters: {non_trainable_params:,} ({non_trainable_params/total_params:.2%})")
    
    # Optional: Print parameters by layer
    print("\nParameters by layer:")
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel():,} parameters")

# Example usage
print_model_parameters(model_interface.model)

Total parameters: 356,085
Trainable parameters: 356,085 (100.00%)
Non-trainable parameters: 0 (0.00%)

Parameters by layer:
layers.0.norm.weight: 60 parameters
layers.0.norm.bias: 60 parameters
layers.0.linear.weight: 3,840 parameters
layers.0.linear.bias: 64 parameters
layers.1.norm.weight: 64 parameters
layers.1.norm.bias: 64 parameters
layers.1.linear.weight: 8,192 parameters
layers.1.linear.bias: 128 parameters
layers.2.norm.weight: 128 parameters
layers.2.norm.bias: 128 parameters
layers.2.linear.weight: 32,768 parameters
layers.2.linear.bias: 256 parameters
layers.3.norm.weight: 256 parameters
layers.3.norm.bias: 256 parameters
layers.3.linear.weight: 131,072 parameters
layers.3.linear.bias: 512 parameters
layers.4.norm.weight: 512 parameters
layers.4.norm.bias: 512 parameters
layers.4.linear.weight: 131,072 parameters
layers.4.linear.bias: 256 parameters
layers.5.norm.weight: 256 parameters
layers.5.norm.bias: 256 parameters
layers.5.linear.weight: 32,768 parameters
layers.5.lin